In [ ]:
"""
================================================================
        The Fermi-Pasta-Ulam-Tsingou (FPUT) Problem (1D)
================================================================
"""
import glob
from math import log10
from matplotlib.animation import FuncAnimation
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi as PI
import os
import re
from scipy.interpolate import interp1d

#   Improve performance by applying 'jit' decorator
from numba import jit

__version__ = '1.0'

#   Color-friendly colors!
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
plt.rcParams['axes.prop_cycle'] = plt.cycler(color = CB_color_cycle)
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", CB_color_cycle)
del CB_color_cycle

################################################################################
#                                                                              #
#                            CONSTANTS & PARAMETERS                            #
#                                                                              #
################################################################################

N, ALPHA, BETA, N_MODES, config_str, dt, num_steps, time_points, y0_amp, frame_step = (None,) * 10

def init_constants(dt_mantissa: float, dt_exp: int) -> None:
    """
    Initializes constants and parameters (parameters used for time_step_analysis)

    :param  dt_mantissa:    Mantissa of dt (float)
    :param  dt_exp:         Exponent of dt (int)
    """
    global N, ALPHA, BETA, N_MODES, config_str, dt, time_points, y0_amp, frame_step

    N = 2 ** 6                                      #   Number of masses

    ALPHA = 1.0                                     #   Linear spring constant
    BETA = 1e-1                                     #   Nonlinear spring constant
    # BETA = 0                                        #   No Nonlinear term

    N_MODES = 1                                     #   Number of initial modes

    t_max = 1e4                                     #   Maximum time

    config_str = f"{int(N)}_{str(ALPHA).replace('.', "d")}_{str(BETA).replace('.', "d")}_{int(N_MODES)}"
    config_str += f"_{int(log10(t_max))}_{str(dt_mantissa).replace('.', "d")}({int(dt_exp)})"

    dt = dt_mantissa * 10 ** dt_exp                 #   Time step
    
    num_steps = int(np.ceil(t_max / dt)) + 1        #   Number of time steps
    time_points = np.linspace(0, t_max, num_steps)
    y0_amp = 1.0                                    #   Initial displacement amplitude
                                                    #   for the first and last points

    #   ANIMATION PARAMETERS
    if num_steps // (30 * 60) == 0:
        frame_step = 1
    else:
        frame_step = num_steps // (300 * 60)      #   To ensure the GIF has 18000 frames

    return None

init_constants(2.5, -2)

#   Fixed boundary
@jit
def oneD_fixed(y, v):
    """
    List of ODEs of dvdt following fixed boundary conditions; dydt are all v
    
    :param  y:      numpy array of y values
    :param  v:      numpy array of v values
    
    :return v:      numpy array of dydt values
    :return dvdt:   numpy array of dvdt values
    """
    dvdt = np.zeros_like(y)             #   Initialize dvdt
    
    #   Interior particles
    dvdt[1:N - 1] = ALPHA * (y[2:N] + y[:N - 2] - 2 * y[1:N - 1]) * (1 + BETA * (y[2:N] - y[:N - 2]))

    #   Boundary particles
    dvdt[0] = ALPHA * (y[1] - 2 * y[0]) * (1 + BETA * y[1])
    dvdt[N - 1] = ALPHA * (y[N - 2] - 2 * y[N - 1]) * (1 + BETA * (- y[N - 2]))

    return v, dvdt

#   Runge-Kutta 4th order method
@jit
def rk4_step(y, v, dt, func):
    """
    Runge-Kutta 4th order ODE Solver
    
    :param  y:      numpy array of y values
    :param  v:      numpy array of v values
    :param  dt:     timestep
    :param  func:   ODE expression

    :return y_next: numpy array of successive y values
    :return v_next: numpy array of successive v values
    """
    k1_y, k1_v = func(y, v)
    k2_y, k2_v = func(y + dt / 2 * k1_y, v + dt / 2 * k1_v)
    k3_y, k3_v = func(y + dt / 2 * k2_y, v + dt / 2 * k2_v)
    k4_y, k4_v = func(y + dt * k3_y, v + dt * k3_v)
    
    y_next = y + dt / 6 * (k1_y + 2 * k2_y + 2 * k3_y + k4_y)
    v_next = v + dt / 6 * (k1_v + 2 * k2_v + 2 * k3_v + k4_v)
    
    return y_next, v_next

#   Initialized to n modes excitation
def initialize_n_mode(N, mode_no, amp = y0_amp):
    """
    Initialize the system for single mode excitation

    :param  N:      number of particles
    :param  mode_no number of modes
    :param  amp:    excitation amplitude

    :return y:      numpy array of displacements
    :return v:      numpy array of velocities
    """
    n = np.linspace(1, N, N)
    
    y = amp * np.sin(mode_no * np.pi * n / (N + 1))     #   Initialize displacements as
                                                        #   the nth mode sine wave
    v = np.zeros(N)                                     #   Initialize velocities to zero
    
    return y, v

#   Initialized to localized excitation
@jit
def initilize_localized(N, mode_no, amp = y0_amp):
    """
    Initialize the system for localized excitation

    :param  N:          number of particles
    :param  mode_no:    number of modes
    :param  amp:        excitation amplitude

    :return y:      numpy array of displacements
    :return v:      numpy array of velocities
    """
    #   Initial conditions
    y = np.zeros(N)                 #   Displacements
    v = np.zeros(N)                 #   Velocities

    y[0] = amp                      #   Initial displacements
    y[N - 1] = amp
    
    return y, v

def oneD_simulator(ode_solver, init_func, mode, expression):
    """
    Simulates time evolution of the oscillators

    :param  ode_solver: ODE Solving algorithm
    :param  init_func:  type of initilization
    :param  mode:       number of modes
    :param  expression: ODE expression
    
    :return d:          numpy array of displacements for each timestep
    :return v:          numpy array of velocities for each timestep
    """
    #   Initial conditions
    y, v = init_func(N, mode)

    #   Time evolution
    displacements = [y.copy()]      #   2D array of displacements at each time step
    velocities = [v.copy()]         #   2D array of velocities at each time step
    for _ in time_points:
        y, v = ode_solver(y, v, dt, expression)
        displacements.append(y.copy())
        velocities.append(v.copy())

    return np.array(displacements), np.array(velocities)

################################################################################
#                                                                              #
#                          ABOVE CODE USED FOR SET UP                          #
#                                                                              #
################################################################################

#   Simulate FPUT
displacements_1D_fixed, velocities_1D_fixed = oneD_simulator(rk4_step, initialize_n_mode, N_MODES, oneD_fixed)
